In [2]:
import openai
from dotenv import load_dotenv
import os
import random
import re

# ===== 환경 설정 =====
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()

TOTAL_QUESTIONS = 3

SKIP_KEYWORDS = [
    "다음 질문", "다음질문", "스킵", "skip", "pass", "next",
    "next question", "넘어가기", "넘어가자", "넘어갈게요"
]

# 태도/모범/성실 칭찬만 제거
EXCLUDE_KEYWORDS = [
    "성실", "성실함", "성실하게", "모범", "모범적인",
    "태도", "태도가", "자세가 좋", "정성을 다해",
    "발표한 모습이 보기 좋았", "발표를 잘함", "발표를 잘 하",
    "참여도가 높", "열심히 참여", "적극적으로 참여",
    "친구들과 잘 지내", "봉사정신이 투철", "예의바른 태도"
]

# 희망 분야 매핑
CAREER_SUBJECT_MAP = {
    "공학": ["수학", "기하", "미적", "미적분", "과학", "물리",
           "융합과학", "정보", "프로그래밍", "공학", "인공지능 수학"],
    "자연": ["생명", "생명과학", "화학", "지구", "지구과학", "물리", "과학"],
    "의학": ["생명", "생명과학", "화학", "보건", "의학"],
    "컴퓨터": ["정보", "프로그래밍", "AI", "데이터", "수학",
            "기하", "미적분", "인공지능 수학", "컴퓨터공학"],
    "소프트웨어": ["정보", "프로그래밍", "AI", "컴퓨터", "수학"],
    "AI": ["정보", "프로그래밍", "AI", "융합과학", "수학", "인공지능 수학"],
    "상경": ["경제", "사회", "정치", "수학", "확률과통계"],
    "경영": ["경영", "경제", "사회", "수학"],
    "인문": ["국어", "문학", "독서", "사회", "윤리", "철학"],
    "교육": ["교육", "심리", "국어", "사회"]
}

##########################################################
# 1. 전체 텍스트 합치기
##########################################################
def get_full_text(student_data):
    records = student_data.get("academic_records", [])
    if isinstance(records, list):
        full = "\n".join(str(x) for x in records)
    else:
        full = str(records)

    reading = student_data.get("reading", "")
    if reading:
        full += "\n" + str(reading)

    return full


##########################################################
# 2. 희망분야 → 학년별 자동 배정
##########################################################
def extract_career_by_grade(full_text):
    matches = re.findall(r"희망\s*분야\s*([^\n]+)", full_text)

    grade_raw = {1: None, 2: None, 3: None}
    for idx, raw in enumerate(matches[:3], start=1):
        cleaned = raw.replace("분야", "").replace("계열", "").strip()
        grade_raw[idx] = cleaned

    def normalize(field):
        if not field:
            return ""
        if "컴퓨터" in field or "소프트웨어" in field:
            return "컴퓨터"
        if "ai" in field.lower():
            return "AI"
        if "공학" in field:
            return "공학"
        if "자연" in field:
            return "자연"
        if "의학" in field:
            return "의학"
        if "경영" in field:
            return "경영"
        if "상경" in field:
            return "상경"
        if "인문" in field:
            return "인문"
        if "교육" in field:
            return "교육"
        return field

    grade_norm = {g: normalize(v) for g, v in grade_raw.items()}
    return grade_raw, grade_norm


##########################################################
# 3. 세특/창체 출처 자동 추출
##########################################################
def extract_sources(full_text):
    sources = []

    # 3-1) 세부능력특기사항
    pattern = re.compile(
        r"([가-힣A-Za-z0-9\s]+):\s*(.+?)(?=\n[가-힣A-Za-z0-9\s]+:|\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
        re.DOTALL
    )

    for m in pattern.finditer(full_text):
        subject = m.group(1).strip()
        desc = m.group(2).strip().replace("\n", " ")[:250]
        label = f"{subject}(세부능력특기사항)"
        sources.append((label, desc))

    # 3-2) 창체
    blocks = ["동아리활동", "자율활동", "진로활동", "봉사활동"]
    for b in blocks:
        bpat = re.compile(
            b + r"\s*\n(.+?)(?=\n동아리활동|\n자율활동|\n진로활동|\n봉사활동|\Z)",
            re.DOTALL
        )
        for m in bpat.finditer(full_text):
            desc = m.group(1).strip().replace("\n", " ")[:250]
            sources.append((b, desc))

    return sources


##########################################################
# 4. 태도성 내용 제거
##########################################################
def filter_out_attitude(sources):
    clean = []
    for label, text in sources:
        combo = label + " " + text
        if not any(bad in combo for bad in EXCLUDE_KEYWORDS):
            clean.append((label, text))
    return clean


##########################################################
# 5. label → 과목명/활동종류 분리
##########################################################
def split_label(label):
    """
    label 예:
    - '인공지능 수학(세부능력특기사항)'
    - '동아리활동'
    """
    if "(" in label:
        subject = label.split("(")[0]
        activity = label[label.find("(")+1:-1]
    else:
        subject = label
        activity = "창체활동"

    return subject.strip(), activity.strip()


##########################################################
# 6. 메인 로직
##########################################################
def start_ai_interview(student_data):
    full_text = get_full_text(student_data)

    # --- 희망분야 추출 ---
    career_raw, career_norm = extract_career_by_grade(full_text)

    # --- 출처 추출 ---
    sources = extract_sources(full_text)
    sources = filter_out_attitude(sources)

    if not sources:
        print("출처 없음. JSON 구조 확인 필요.")
        return

    # --- A 모드 SYSTEM PROMPT ---
    system_prompt = '''
너는 대한민국 최상위권 공대 면접관이다.
매우 냉정하고 날카롭게 평가하며 생기부와 무관한 답변은 모두 혹평하라.
점수 < 70점이면 반드시 [다시 답변 요청]을 붙인다.
'''

    messages = [{"role": "system", "content": system_prompt}]

    question_num = 1

    while question_num <= TOTAL_QUESTIONS:

        # === 1) 학년 랜덤 선택 ===
        selected_grade = random.choice([1,2,3])
        selected_career_raw = career_raw[selected_grade]
        selected_career_norm = career_norm[selected_grade]

        # === 2) 해당 학년 희망분야 기반 필터 ===
        allowed_keywords = CAREER_SUBJECT_MAP.get(selected_career_norm, [])
        grade_sources = [
            s for s in sources if any(k in (s[0] + s[1]) for k in allowed_keywords)
        ]
        if not grade_sources:
            grade_sources = sources

        # === 3) 출처 선택 ===
        label, text = random.choice(grade_sources)
        subject_name, activity_type = split_label(label)

        is_last = (question_num == TOTAL_QUESTIONS)

        # === 4) 질문 생성 ===
        user_prompt = f'''
다음 정보를 기반으로 {"[마지막 질문]" if is_last else "[질문]"}을 생성하라.

출처 학년: {selected_grade}학년
과목명: {subject_name}
활동종류: {activity_type}
출처 전문: {label}
핵심 내용: {text}

해당 학년 희망분야: {selected_career_raw} (키워드: {selected_career_norm})

형식:
{"[마지막 질문]" if is_last else "[질문]"}
출처: {selected_grade}학년 · {subject_name} ({activity_type})
희망분야({selected_grade}학년): {selected_career_raw}
핵심 내용: {text}
질문:
'''
        messages.append({"role": "user", "content": user_prompt})

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=600
        )
        qtext = resp.choices[0].message.content
        print("\n" + qtext)
        messages.append({"role": "assistant", "content": qtext})

        # === 5) 답변 받기 ===
        answer = input("\n[학생 답변 또는 '다음 질문'] > ")

        if any(k in answer.lower() for k in SKIP_KEYWORDS):
            print("\n[안내] 다음 질문으로 넘어갑니다.\n")
            question_num += 1
            continue

        if answer.lower() in ("exit", "quit"):
            print("면접을 종료합니다.")
            break

        # === 6) 평가 ===
        eval_prompt = f'''
[학생 답변]
{answer}

출처 학년: {selected_grade}학년
과목명: {subject_name}
활동종류: {activity_type}
출처 전문: {label}
핵심 내용: {text}
희망분야({selected_grade}학년): {selected_career_raw}

A 모드로 매우 날카롭게 평가하라.
'''
        messages.append({"role": "user", "content": eval_prompt})

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=900
        )
        eval_text = resp.choices[0].message.content
        print("\n" + eval_text)
        messages.append({"role": "assistant", "content": eval_text})

        if "[다시 답변 요청]" in eval_text:
            continue

        question_num += 1


##########################################################
# 7. 생활기록부 TXT 파일을 기존 코드가 쓰는 데이터 구조로 변환
##########################################################
def load_student_txt(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read().strip()

    # 이름 추출 (없어도 동작)
    name_match = re.search(r"이름\s*[:：]\s*(.+)", text)
    name = name_match.group(1).strip() if name_match else "이름 미상"

    return {
        "student_info": {
            "name": name
        },
        "academic_records": text,  # 문자열 그대로 사용
        "reading": ""
    }


##########################################################
# 실행
##########################################################
RECORDINGS_DIR = "recordings_lite"
TARGET_FILE = "202315060_3학년_휴먼AI공학전공_서현승_정시_censored.txt"


if __name__ == "__main__":

    TARGET_TXT_PATH = os.path.join(RECORDINGS_DIR, TARGET_FILE)  # 👈 여기만 바꾸면 됨

    if not os.path.exists(TARGET_TXT_PATH):
        print(f"❌ 파일을 찾을 수 없습니다: {TARGET_TXT_PATH}")
        exit()

    student_data = load_student_txt(TARGET_TXT_PATH)
    
    print("📄 사용 파일:", TARGET_TXT_PATH)
    print("-" * 60)

    start_ai_interview(student_data)

📄 사용 파일: recordings_lite\202315060_3학년_휴먼AI공학전공_서현승_정시_censored.txt
------------------------------------------------------------

[질문]
출처: 2학년 · 정부24 과목 세 부 능력 및특기사 항 수학 (세부능력특기사항)  
희망분야(2학년): None  
핵심 내용: '00수학저널 활동 (2018.3.2. 2018.7.23.) '의 일환으로   통계활용보고서 '고등학교 1학년의 책 분야 선호도 조사' 틀 연구함. (평가) 학생의 보고서 연구 동기는 요즘 학생들의 삶에 책이 차지하는 공간이 줄어들어 , 학생들에게 관심을 받으면서도 유의할 종 류의 책올 학교에 비치하기 위해 책의 분야률 정리하고 싶엇년 짓임. 보고서 연구내용은 학생들의 각 진로 과정에 따라 선호 하는 특정 분야의 책 , 교사들이 학생들에게 권하  
질문: 이 통계활용보고서를 작성하면서 어떤 연구 방법론을 사용했는지 구체적으로 설명해 주시겠습니까? 그리고 그 연구 방법이 결과에 어떠한 영향을 미쳤다고 생각하나요?



[학생 답변 또는 '다음 질문'] >  고등학교 1학년 친구들이 실제 선호하는 책을 조사하는 방법을 이용했으며, 자료의 맞춤성과 정확성을 높일 수 있었다 생각합니다.



해당 답변은 매우 부족합니다. '실제 선호하는 책을 조사하는 방법'이라는 표현은 포괄적이며 구체적인 방법론에 대한 설명이 결여되어 있습니다. 연구 방법론에서 어떤 구체적인 접근 방식을 사용했는지, 예를 들어 설문조사를 했는지, 인터뷰를 했는지 등에 대한 언급이 없어 신뢰성을 줄 수 없습니다. 또한, "자료의 맞춤성과 정확성을 높일 수 있었다"고 주장했으나, 이를 뒷받침할 만한 근거도 부족합니다. 연구 결과에 대한 구체적인 분석이나 해석이 전혀 없다는 점도 큰 단점입니다.

결론적으로, 연구 방법과 결과 분석에 대한 명확한 설명이 결여되어 있어 이 답변은 70점 미만의 평가를 받을 수밖에 없습니다. 

[다시 답변 요청]

[질문]  
출처: 3학년 · 교양 이수단위 합계 25 25 과목 세 부 능력 및 특기사 항 한국사 (세부능력특기사항)  
희망분야(3학년): None  
핵심 내용: (1학기) 장영실에 대한 보고서 작성함. 자격루와 같은 장영실의 발명품에 대해서 다루기 위해 다양한 참고 자료를 활용하여 자격루가 그 이전의 조선 물시계의 단점을 해결한 방법을 다른 나라의 물시계가 해결한 방법과 비교하고 혼천이가 어떤 우주관을 기반으로 만들어졌는지, 우기의 특이한 형태가 무슨 이유에서 탄생하게 된 것인지 조사함. (2학기) 김홍 집율 주제로 설정한 모의 재판 활동에 중인 역할로 참여함. ‘번역 오하기분’ 등 다  
질문: 장영실의 발명품인 자격루와 관련하여, 그 전의 조선 물시계의 단점은 무엇이었고 자격루는 어떻게 이를 개선했는지 구체적으로 설명해 주시겠습니까? 또한, 혼천의 설계가 어떤 우주관에 근거하고 있는지에 대한 개인적인 해석도 부탁드립니다.



[학생 답변 또는 '다음 질문'] >  그 전 조선 물시계는 물의 높이로 시간을 아는 것이라 정확한 시간을 알기는 힘들었고, 자격루는 이를 물로 기계장치가 움직이게 함으로써 시간을 정확히 표시하게 개선하였습니다. 혼천의는 지구를 중심으로 설계되어 있어 천동설에 근거하고 있다고 생각합니다.


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-ieq3S40n7w9a3M3JXtI33y8c on tokens per min (TPM): Limit 100000, Used 100000, Requested 2252. Please try again in 16h12m51.839s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}